In [1]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import json
from shapely.ops import cascaded_union
from shapely.geometry import Polygon, LineString
from osgeo import ogr

%matplotlib inline

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (16,16)

In [1]:
import geojson


## Portland

In [2]:
portland_sa = gpd.GeoDataFrame.from_file('../cities/portland/urban_edge_t3.geojson')
portland_bus = gpd.GeoDataFrame.from_file('../cities/portland/portland_bus.geojson')

In [ ]:
portland_sa_geom = cascaded_union(portland_sa['geometry'])
portland_bus_geom = cascaded_union(portland_bus['geometry'])

In [ ]:
portland_sa_bus_geom = portland_sa_geom.intersection(portland_bus_geom)

In [17]:
portland_sa_bus_geom

In [20]:
portland_bus_intersect = portland_bus[portland_bus.geometry.intersects(portland_sa.geometry)]

## OGR intersection

In [16]:
#portland_sa_bus_geom.shape_factory?

In [3]:
ogr.UseExceptions()

In [5]:
#GEOJSON_SQL = """SELECT ST_AsText(ST_GeomFromGeoJSON(portland_sa)) as wkt"""

In [4]:
# Reading in Study area as layer
driver = ogr.GetDriverByName('GeoJSON')
ds = ogr.Open('../cities/portland/urban_edge_t3.geojson')
sa_layer = ds.GetLayer('OGRGeoJSON')

In [5]:
#Reading in bus data
bus_ds = ogr.Open('lines.geojson')
bus_layer = bus_ds.GetLayer('OGRGeoJSON')
ogr.GeometryTypeToName(bus_layer.GetGeomType())

'Line String'

In [6]:

    
# create an empty geometry of the same type
union1 = ogr.Geometry(bus_layer.GetGeomType())

# union all the geometrical features of layer 1
for feat in bus_layer:
    geom =feat.GetGeometryRef()
    union1 = union1.Union(geom)
    
# same for layer2
union2 = ogr.Geometry(sa_layer.GetGeomType())
for feat in sa_layer:
    geom =feat.GetGeometryRef()  
    union2 = union2.Union(geom)
    


In [7]:
import os
os.system('say "your program has finished"')


0

In [9]:
intersection = union1.Intersection(union2)

In [10]:
with open('intersection.geojson', 'w') as f:
    f.write(intersection.ExportToJson())
